# Calculating Gazing Score
## Calculate Gazing Score for choosing High Score Block in proposed model update method

# ------------------------------------------------------------------------------------------

## Get Score-CAM output from each block

In [ ]:
#import
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import save_model, load_model

from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.utils import plot_model, to_categorical
from keras.callbacks import TensorBoard

import matplotlib.pyplot as plt
import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import time
from natsort import natsorted

from tensorflow.keras.applications import MobileNet
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import confusion_matrix
import seaborn as sns

from keras.models import Model
from gradcamutils import GradCam, GradCamPlusPlus, ScoreCam, GuidedBackPropagation, superimpose, read_and_preprocess_img, build_guided_model

In [ ]:
#import images for Score-CAM
scorefiles_0 = glob.glob(r"C:\Users\pass")
scorefiles_1 = glob.glob(r"C:\Users\pass")
scorefiles_2 = glob.glob(r"C:\Users\pass")
scorefiles_3 = glob.glob(r"C:\Users\pass")
scorefiles_4 = glob.glob(r"C:\Users\pass")
scorefiles_5 = glob.glob(r"C:\Users\pass")
scorefiles_6 = glob.glob(r"C:\Users\pass")

#sort the files
sorted_files_0 = natsorted(scorefiles_0)
sorted_files_1 = natsorted(scorefiles_1)
sorted_files_2 = natsorted(scorefiles_2)
sorted_files_3 = natsorted(scorefiles_3)
sorted_files_4 = natsorted(scorefiles_4)
sorted_files_5 = natsorted(scorefiles_5)
sorted_files_6 = natsorted(scorefiles_6)

#making list
scorefile_list = []

scorefile_list0 = []
scorefile_list1 = []
scorefile_list2 = []
scorefile_list3 = []
scorefile_list4 = []
scorefile_list5 = []
scorefile_list6 = []

for file in sorted_files_0:
    file = cv2.imread(file)
    scorefile_list0.append(file)

for file in sorted_files_1:
    file = cv2.imread(file)
    scorefile_list1.append(file)
    
for file in sorted_files_2:
    file = cv2.imread(file)
    scorefile_list2.append(file)

for file in sorted_files_3:
    file = cv2.imread(file)
    scorefile_list3.append(file)
    
for file in sorted_files_4:
    file = cv2.imread(file)
    scorefile_list4.append(file)

for file in sorted_files_5:
    file = cv2.imread(file)
    scorefile_list5.append(file)

for file in sorted_files_6:
    file = cv2.imread(file)
    scorefile_list6.append(file)

for file in scorefile_list0:
    scorefile_list.append(file)
for file in scorefile_list1:
    scorefile_list.append(file)
for file in scorefile_list2:
    scorefile_list.append(file)
for file in scorefile_list3:
    scorefile_list.append(file)
for file in scorefile_list4:
    scorefile_list.append(file)
for file in scorefile_list5:
    scorefile_list.append(file)
for file in scorefile_list6:
    scorefile_list.append(file)

#normalization
scorefile_list = [file.astype(float)/255 for file in scorefile_list]
scorefile_list = [cv2.resize(file, (360, 270)) for file in scorefile_list]

#numpy list
rawscorefile_list = scorefile_list
npscorefile_list = np.array(scorefile_list)

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import gc
tf.compat.v1.disable_eager_execution()

def normalize(x):
        """Utility function to normalize a tensor by its L2 norm"""
        return (x + 1e-10) / (K.sqrt(K.mean(K.square(x))) + 1e-10)


def softmax(x):
    f = np.exp(x)/np.sum(np.exp(x), axis = 1, keepdims = True)
    return f

def GradCam(model, img_array, layer_name):
    cls = np.argmax(model.predict(img_array))
    
    """GradCAM method for visualizing input saliency."""
    y_c = model.output[0, cls]
    conv_output = model.get_layer(layer_name).output
    grads = tf.gradients(y_c, conv_output)[0]
    # grads = normalize(grads)

    gradient_function = K.function([model.input], [conv_output, grads])
    output, grads_val = gradient_function([img_array])
    output, grads_val = output[0, :], grads_val[0, :, :, :]
    weights = np.mean(grads_val, axis=(0, 1))

    cam = np.dot(output, weights)
    cam = np.maximum(cam, 0)  # Passing through ReLU
    cam /= np.max(cam)  # scale 0 to 1.0  

    return cam

def GradCamPlusPlus(model, img_array, layer_name):
    cls = np.argmax(model.predict(img_array))
    y_c = model.output[0, cls]
    conv_output = model.get_layer(layer_name).output
    grads = tf.gradients(y_c, conv_output)[0]
    # grads = normalize(grads)

    first = K.exp(y_c)*grads
    second = K.exp(y_c)*grads*grads
    third = K.exp(y_c)*grads*grads*grads

    gradient_function = K.function([model.input], [y_c,first,second,third, conv_output, grads])
    y_c, conv_first_grad, conv_second_grad,conv_third_grad, conv_output, grads_val = gradient_function([img_array])
    global_sum = np.sum(conv_output[0].reshape((-1,conv_first_grad[0].shape[2])), axis=0)

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum.reshape((1,1,conv_first_grad[0].shape[2]))
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, np.ones(alpha_denom.shape))
    alphas = alpha_num/alpha_denom

    weights = np.maximum(conv_first_grad[0], 0.0)
    alpha_normalization_constant = np.sum(np.sum(alphas, axis=0),axis=0)
    alphas /= alpha_normalization_constant.reshape((1,1,conv_first_grad[0].shape[2]))
    deep_linearization_weights = np.sum((weights*alphas).reshape((-1,conv_first_grad[0].shape[2])),axis=0)

    cam = np.sum(deep_linearization_weights*conv_output[0], axis=2)
    cam = np.maximum(cam, 0)  # Passing through ReLU
    cam /= np.max(cam) # scale 0 to 1.0  

    return cam


def ScoreCam(model, img_array, layer_name, max_N=-1):

    cls = np.argmax(model.predict(img_array))
    #print(cls)
    act_map_array = Model(inputs=model.input, outputs=model.get_layer(layer_name).output).predict(img_array)
    
    # extract effective maps
    if max_N != -1:
        act_map_std_list = [np.std(act_map_array[0,:,:,k]) for k in range(act_map_array.shape[3])]
        unsorted_max_indices = np.argpartition(-np.array(act_map_std_list), max_N)[:max_N]
        max_N_indices = unsorted_max_indices[np.argsort(-np.array(act_map_std_list)[unsorted_max_indices])]
        act_map_array = act_map_array[:,:,:,max_N_indices]

    input_shape = model.layers[0].output_shape[0][1:]  # get input shape
    # 1. upsampled to original input size
    act_map_resized_list = [cv2.resize(act_map_array[0,:,:,k], input_shape[:2], interpolation=cv2.INTER_LINEAR) for k in range(act_map_array.shape[3])]
    # 2. normalize the raw activation value in each activation map into [0, 1]
    act_map_normalized_list = []
    for act_map_resized in act_map_resized_list:
        if np.max(act_map_resized) - np.min(act_map_resized) != 0:
            act_map_normalized = act_map_resized / (np.max(act_map_resized) - np.min(act_map_resized))
            act_map_normalized.resize(270, 360)
        else:
            act_map_normalized = act_map_resized
            act_map_normalized.resize(270, 360)
        act_map_normalized_list.append(act_map_normalized)
    # 3. project highlighted area in the activation map to original input space by multiplying the normalized activation map
    masked_input_list = []
    for act_map_normalized in act_map_normalized_list:
        masked_input = np.copy(img_array)
        for k in range(3):
            masked_input[0,:,:,k] *= act_map_normalized
        masked_input_list.append(masked_input)
    masked_input_array = np.concatenate(masked_input_list, axis=0)
    # 4. feed masked inputs into CNN model and softmax
    pred_from_masked_input_array = softmax(model.predict(masked_input_array))
    # 5. define weight as the score of target class
    weights = pred_from_masked_input_array[:,cls]
    # 6. get final class discriminative localization map as linear weighted combination of all activation maps
    cam = np.dot(act_map_array[0,:,:,:], weights)
    cam = np.maximum(0, cam)  # Passing through ReLU
    cam /= np.max(cam)  # scale 0 to 1.0
    
    return cam

def superimpose(img_array, cam, emphasize=False):
    
    img_bgr = img_array

    heatmap = cv2.resize(cam, (img_bgr.shape[1], img_bgr.shape[0]))
    if emphasize:
        heatmap = sigmoid(heatmap, 50, 0.5, 1)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    hif = .8
    superimposed_img = heatmap * hif + img_bgr
    superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    superimposed_img_rgb = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    
    return superimposed_img_rgb

import tensorflow.keras

import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.keras.applications.vgg16 import preprocess_input

def sigmoid(x, a, b, c):
    return c / (1 + np.exp(-a * (x-b)))


#from tensorflow.keras.preprocessing.image import load_img, img_to_array

def read_and_preprocess_img(image):
    x = image
    x = np.expand_dims(x, axis=0)
    #x = preprocess_input(x)
    return x

In [ ]:
model = load_model('BaseModel.h5')

for i in range(len(scorefile_list)):
    index = i
    raw_image = rawscorefile_list[index]
    image = scorefile_list[index]
    img_array = read_and_preprocess_img(image)
    layer_name = "conv_pw_1_relu"
    score_cam=ScoreCam(model,img_array,layer_name)
    scorecam_rescaled = cv2.resize(score_cam, (360, 270))
    scorecam_rescaled = (scorecam_rescaled * 255).astype(np.uint8)  
    cv2.imwrite('saved_image' + str(index) + '.jpg', scorecam_rescaled)

# ------------------------------------------------------------------------------------------

## Calculate Gazing Score using Score-CAM result got above

In [ ]:
import matplotlib.pyplot as plt
import glob
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import random
import re
from natsort import natsorted

In [ ]:
#get processed images(these images are processed to mask images)
files_0 = glob.glob(r"C:\Users\pass")
files_1 = glob.glob(r"C:\Users\pass")
files_2 = glob.glob(r"C:\Users\pass")
files_3 = glob.glob(r"C:\Users\pass")
files_4 = glob.glob(r"C:\Users\pass")
files_5 = glob.glob(r"C:\Users\pass")
files_6 = glob.glob(r"C:\Users\pass")

#sort
sorted_files_0 = natsorted(files_0)
sorted_files_1 = natsorted(files_1)
sorted_files_2 = natsorted(files_2)
sorted_files_3 = natsorted(files_3)
sorted_files_4 = natsorted(files_4)
sorted_files_5 = natsorted(files_5)
sorted_files_6 = natsorted(files_6)

file_list = []

file_list0 = []
file_list1 = []
file_list2 = []
file_list3 = []
file_list4 = []
file_list5 = []
file_list6 = []

for file in sorted_files_0:
    file = cv2.imread(file)
    file_list0.append(file)

for file in sorted_files_1:
    file = cv2.imread(file)
    file_list1.append(file)
    
for file in sorted_files_2:
    file = cv2.imread(file)
    file_list2.append(file)

for file in sorted_files_3:
    file = cv2.imread(file)
    file_list3.append(file)
    
for file in sorted_files_4:
    file = cv2.imread(file)
    file_list4.append(file)

for file in sorted_files_5:
    file = cv2.imread(file)
    file_list5.append(file)

for file in sorted_files_6:
    file = cv2.imread(file)
    file_list6.append(file)

for file in file_list0:
    file_list.append(file)
for file in file_list1:
    file_list.append(file)
for file in file_list2:
    file_list.append(file)
for file in file_list3:
    file_list.append(file)
for file in file_list4:
    file_list.append(file)
for file in file_list5:
    file_list.append(file)
for file in file_list6:
    file_list.append(file)
    
print(len(file_list))

In [ ]:
# definition of color range of red

lower_red1 = np.array([0, 50, 50])   
upper_red1 = np.array([10, 255, 255]) 
lower_red2 = np.array([170, 50, 50])  
upper_red2 = np.array([180, 255, 255]) 

right_score = 1.0
fault_score = -0.1

#list for mask images
mask_list = []

#make mask images
for img in file_list:

    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    mask1 = cv2.inRange(hsv_img, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv_img, lower_red2, upper_red2)
    mask = cv2.bitwise_or(mask1, mask2)  

    processed_img = np.full(img.shape[:2], fault_score)  
    processed_img[mask > 0] = right_score  
    mask_list.append(processed_img)

mask_list = [cv2.resize(file, (360, 270)) for file in mask_list]

In [ ]:
#import score-cam result
files_block1 = glob.glob(r"C:\Users\pass")
files_block2 = glob.glob(r"C:\Users\pass")
files_block3 = glob.glob(r"C:\Users\pass")
files_block4 = glob.glob(r"C:\Users\pass")
files_block5 = glob.glob(r"C:\Users\pass")
files_block6 = glob.glob(r"C:\Users\pass")
files_block7 = glob.glob(r"C:\Users\pass")
files_block8 = glob.glob(r"C:\Users\pass")
files_block9 = glob.glob(r"C:\Users\pass")
files_block10 = glob.glob(r"C:\Users\pass")
files_block11 = glob.glob(r"C:\Users\pass")
files_block12 = glob.glob(r"C:\Users\pass")
files_block13 = glob.glob(r"C:\Users\pass")


# sort
sorted_block1 = natsorted(files_block1)
sorted_block2 = natsorted(files_block2)
sorted_block3 = natsorted(files_block3)
sorted_block4 = natsorted(files_block4)
sorted_block5 = natsorted(files_block5)
sorted_block6 = natsorted(files_block6)
sorted_block7 = natsorted(files_block7)
sorted_block8 = natsorted(files_block8)
sorted_block9 = natsorted(files_block9)
sorted_block10 = natsorted(files_block10)
sorted_block11 = natsorted(files_block11)
sorted_block12 = natsorted(files_block12)
sorted_block13 = natsorted(files_block13)

file_list_block1 = []
file_list_block2 = []
file_list_block3 = []
file_list_block4 = []
file_list_block5 = []
file_list_block6 = []
file_list_block7 = []
file_list_block8 = []
file_list_block9 = []
file_list_block10 = []
file_list_block11 = []
file_list_block12 = []
file_list_block13 = []


for file in sorted_block1:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block1.append(file)

for file in sorted_block2:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block2.append(file)
    
for file in sorted_block3:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block3.append(file)

for file in sorted_block4:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block4.append(file)
    
for file in sorted_block5:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block5.append(file)

for file in sorted_block6:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block6.append(file)

for file in sorted_block7:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block7.append(file)

for file in sorted_block8:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block8.append(file)

for file in sorted_block9:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block9.append(file)
    
for file in sorted_block10:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block10.append(file)

for file in sorted_block11:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block11.append(file)
    
for file in sorted_block12:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block12.append(file)

for file in sorted_block13:
    file = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    file_list_block13.append(file)


In [ ]:
#fuctions for calculating Gazing Score from mask image and score-cam result
def combine_images(mask_image, grayscale_image, right_score, fault_score):
    mask_ones = mask_image == right_score
    mask_negative_ones = mask_image == fault_score
    
    combined_ones = mask_ones * grayscale_image
    combined_negative_ones = mask_negative_ones * grayscale_image * fault_score
    
    sum_ones = np.sum(combined_ones)
    sum_negative_ones = np.sum(combined_negative_ones)
    
    count_ones = np.sum(mask_ones)
    count_negative_ones = np.sum(mask_negative_ones)
    
    avg_ones = sum_ones / count_ones if count_ones > 0 else 0
    avg_negative_ones = sum_negative_ones / count_negative_ones if count_negative_ones > 0 else 0
    
    return avg_ones + avg_negative_ones

def process_multiple_images(mask_list, file_list_block):
    results = []
    
    if len(mask_list) != len(file_list_block):
        raise ValueError("the lengths of mask_list and file_list_block are different")
    
    for i in range(len(mask_list)):
        mask_image = mask_list[i]
        grayscale_image = file_list_block[i]
        
        result = combine_images(mask_image, grayscale_image, right_score, fault_score)
        
        results.append(result)
    
    return results

In [ ]:
#calculate Gazing Score for each block
result_block1 = process_multiple_images(mask_list, file_list_block1)
result_block2 = process_multiple_images(mask_list, file_list_block2)
result_block3 = process_multiple_images(mask_list, file_list_block3)
result_block4 = process_multiple_images(mask_list, file_list_block4)
result_block5 = process_multiple_images(mask_list, file_list_block5)
result_block6 = process_multiple_images(mask_list, file_list_block6)
result_block7 = process_multiple_images(mask_list, file_list_block7)
result_block8 = process_multiple_images(mask_list, file_list_block8)
result_block9 = process_multiple_images(mask_list, file_list_block9)
result_block10 = process_multiple_images(mask_list, file_list_block10)
result_block11 = process_multiple_images(mask_list, file_list_block11)
result_block12 = process_multiple_images(mask_list, file_list_block12)
result_block13 = process_multiple_images(mask_list, file_list_block13)

In [ ]:
import csv

data = [result_block1, result_block2, result_block3, result_block4, result_block5, result_block6, result_block7, result_block8, result_block9, result_block10, result_block11, result_block12, result_block13]

# CSV output
def save_to_csv(data, filename="whichlayer.csv"):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        for row in data:
            writer.writerow(row)

# save
save_to_csv(data, "whichlayer.csv")